In [18]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

In [19]:
df = pd.read_parquet('../../../metrics.parquet')

In [20]:
df, _ = train_test_split(df, train_size=200000, stratify=df['time_to_stop_activity'], random_state=11)

In [21]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [22]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [23]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [24]:
def train_evaluate_bayesian(alpha_1, alpha_2, lambda_1, lambda_2, X_train, Y_train, X_test, Y_test):
    mse = None; mae = None; r2 = None

    try:
        bayesian_ridge = BayesianRidge(
            alpha_1=alpha_1,
            alpha_2=alpha_2,
            lambda_1=lambda_1,
            lambda_2=lambda_2,
        )

        bayesian_ridge.fit(X_train, Y_train)
        y_pred = bayesian_ridge.predict(X_test)

        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(f'Bayesian Ridge - Alpha1: {alpha_1}, Alpha2: {alpha_2}, Lambda1: {lambda_1}, Lambda2: {lambda_2} Finalized - {mse}, {mae}, {r2}')

        return {
            'alpha_1': alpha_1,
            'alpha_2': alpha_2,
            'lambda_1': lambda_1,
            'lambda_2': lambda_2,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }

    except Exception as e:
        print(f'Bayesian Ridge - Alpha1: {alpha_1}, Alpha2: {alpha_2}, Lambda1: {lambda_1}, Lambda2: {lambda_2} Error: {e}')
        
        return {
            'alpha_1': alpha_1,
            'alpha_2': alpha_2,
            'lambda_1': lambda_1,
            'lambda_2': lambda_2,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }

In [25]:
param_combinations = [
    (alpha_1, alpha_2, lambda_1, lambda_2)
    for alpha_1 in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    for alpha_2 in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    for lambda_1 in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    for lambda_2 in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
]

In [26]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [28]:
benchmark = Parallel(n_jobs=-1)(
    delayed(train_evaluate_bayesian)(alpha_1, alpha_2, lambda_1, lambda_2, X_train, Y_train, X_test, Y_test)
    for alpha_1, alpha_2, lambda_1, lambda_2 in param_combinations
)

Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-05, Lambda2: 1e-06 Finalized - 9.726372344528693, 1.9498124889465933, 0.43927611085201523
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-06, Lambda2: 1e-05 Finalized - 9.726372341049942, 1.9498124887112476, 0.4392761110525647
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-06, Lambda2: 0.01 Finalized - 9.726372340508949, 1.9498124886746488, 0.43927611108375286
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-06, Lambda2: 0.1 Finalized - 9.726372335635137, 1.9498124883449246, 0.43927611136472733
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-05, Lambda2: 0.0001 Finalized - 9.726372344523309, 1.949812488946229, 0.43927611085232554
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-05, Lambda2: 0.01 Finalized - 9.726372343987194, 1.9498124889099597, 0.4392761108832326
Bayesian Ridge - Alpha1: 1e-06, Alpha2: 1e-06, Lambda1: 1e-06, Lambda2: 1e-06 Finalized - 9.726372341050409, 1.9

In [29]:
benchmark_df = pd.DataFrame(benchmark)

In [30]:
benchmark_df.to_excel('BayesianRegressorBenchmark.xlsx')